# Connect or Collect

Este Notebook muestra el pardiagma "Connect or Collect" que permite incorporar datos externos en los analysis de IRIS sin necesidad de copiar los datos, mediante la funcionalidad de <b>Foreign Tables</b>

- el CRM exporta la información de clientes en formato CSV. Accedimos a ello sin copia con Foreign Table
- La tienda online mantiene la información de pedidos en su BBDD. Accedimos a ello sin copia con Foreign Table
- La nueva empresa adquerida exporta CSV. Los importamos en IRIS con el Data Mapper de la capa de Interoperabilidad


<img src="./img/scenario.png" style="height:300px">

## Initialización

In [ ]:
#Some imports
import iris,time,os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set an environment variable
# os.environ['CLASSPATH'] = '/notebooks/intersystems-jdbc-3.9.0.jar'
# os.environ['JAVA_HOME']= 'usr/lib/jvm/java-11-openjdk'

## Conectar a IRIS

In [ ]:
# Use JDBC and JayDeBeAPI to connect and run SQL (IRIS DB-API only allows SQL DML)
import jaydebeapi
url = "jdbc:IRIS://iris:1972/IRISAPP"
driver = "com.intersystems.jdbc.IRISDriver"
user = "SuperUser"
password = "SYS"
jarfile = "/notebooks/intersystems-jdbc-3.9.0.jar"

In [ ]:
connection = jaydebeapi.connect(driver, url, [user, password], jarfile)


In [ ]:
# Define Execute function
def exec_sql(sql):
    c1=connection.cursor()
    c1.execute(sql)

#Define Query execution function
def just_sql(sql):
    c1 = connection.cursor()
    c1.execute(sql)
    result = c1.fetchall()[0]
    c1.close()
    return result  

# Paradigma CONNECT

En el paradigma Connect accede a los datos in situ sin realizar ninguna copia, para casos donde no es conveniente importar los datos en IRIS.

## Referenciar Ficheros Externos

In [ ]:
#Drop Foreign Server if already Exist
#exec_sql("DROP FOREIGN SERVER ext.crm CASCADE")

In [ ]:
# Create Foreign Server
exec_sql("CREATE FOREIGN SERVER ext.crm FOREIGN DATA WRAPPER CSV HOST '/data/crm/'")

In [ ]:
# Create the Foreign Table
exec_sql('''CREATE FOREIGN TABLE crm.clientes( 
    IDCliente VARCHAR(30),
    Nombre VARCHAR(50),
    Apellidos  VARCHAR(50),
    Genero VARCHAR(1),
    FechaNacimiento DATE,
    DireccionPais VARCHAR(20),
    DireccionCiudad VARCHAR(20),
    DireccionRegion VARCHAR(20),
    DireccionCodigoPostal VARCHAR(10)) SERVER ext.crm FILE 'clientes.csv' 
    USING '{"from":{"file":{"header":true}}}' ''')

In [ ]:
#Get and Display the Contents
df=pd.read_sql("select top 5 * from crm.clientes",connection)
display(df)

# Referenciar BBDD Externa

Para referenciar una base de datos externa, primero se tiene que instalar el driver odbc/jdbc correspondiente en el servidor/VM/docker de IRIS y definir una conexión de SQLGateway como se muestra aquí.

<img src="./img/sqlgatewayconnection.png" style="height:300px">

El Foreign Server referencia la conexión de SQL Gateway

In [ ]:
#Cleanup: drop Foreing Server and Defined Tables if already exist
#exec_sql("DROP FOREIGN SERVER ext.onlinestore CASCADE")

In [ ]:
#Create Foreign Server
exec_sql("create foreign server ext.onlinestore foreign data wrapper jdbc connection 'onlinestore'")

In [ ]:
#Create Foreign Table
exec_sql('''
    CREATE FOREIGN TABLE onlinestore.orders(
    id bigint,
    invoice VARCHAR(20),
    stockcode VARCHAR(20),
    description VARCHAR(100),
    quantity INTEGER,
    invoicedate Date,
    price NUMERIC,
    customerid VARCHAR(10)
    --,country VARCHAR(50)
    ) server ext.onlinestore table 'public.orders'
''')

In [ ]:
#Get and Display the Contents
df=pd.read_sql("select top 5 * from onlinestore.orders",connection)
display(df)

# Juntar los Datos

Ahora podemos executar Queries SQL que hagan Joins entre las tablas de los distintos Foreign servers.

In [ ]:
sql=''' select o.invoice, o.stockcode, o.description, o.customerid, 
        c.IDCliente, c.nombre, c.apellidos
        FROM onlinestore.orders o LEFT OUTER JOIN crm.clientes c
        on o.customerid = c.IDCliente
        where o.customerid in ('17850','13047','12853')
    '''
df=pd.read_sql(sql,connection)
display(df)

In [ ]:
#Sales by Gender
sql=''' select SUM (o.quantity) as total_quantity, c.genero as genero
        FROM onlinestore.orders o LEFT OUTER JOIN crm.clientes c
        on o.customerid = c.IDCliente
        group by c.genero
    '''
df=pd.read_sql(sql,connection)
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the Seaborn theme for better visuals
sns.set_theme(style="whitegrid")

# Create the bar chart
plt.figure(figsize=(8, 6))
sns.barplot(data=df, x='genero', y='total_quantity', palette="viridis")

# Add labels and a title
plt.title("Total Quantity by Genero", fontsize=16)
plt.xlabel("Genero", fontsize=14)
plt.ylabel("Total Quantity", fontsize=14)

# Show the plot
plt.show()

# Foreign Server: Queries Passthrough

Es posible enviar queries SQL a un foreign server en modo "passthrough", directamente con la sinytaxis SQL del motor remoto

In [ ]:
df=pd.read_sql("THROUGH foreign server ext.onlinestore select * from public.orders LIMIT 2",connection)
display(df)

# Paradigma COLLECT

En el paradigma collect, se realiza una carga de datos. El comando SQL "LOAD DATA" permite hace importaciones massivas de datos externos limpios, sea desde fichero o servidor externo accesible por JDBC.
Este mecanismo usa internamente el Java Gateway.

## Carga desde Ficheros Externos

Requiere definir la tabla donde cargar los datos, y mapear todo los campos del fichero.

In [ ]:
# Prepare the target Table. The table needs to be created
# /data/superstore/superstore.csv
# Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,
#  State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit

#exec_sql('Drop table bulk.superstore')

sql=''' CREATE TABLE bulk.superstore (
    OrderID VARCHAR(30),OrderDate VARCHAR(30), 
    CustomerID VARCHAR(30),
    Category VARCHAR(30),
    ProductName VARCHAR(100),
    Sales DOUBLE,
    Quantity INTEGER)
    '''
exec_sql(sql)

In [ ]:
sql=''' LOAD DATA FROM FILE '/data/superstore/superstore.csv'
    COLUMNS ( "Row ID" VARCHAR(20),
    "Order ID" VARCHAR(30) ,"Order Date" VARCHAR(30), "Ship Date" VARCHAR(30),"Ship Mode" VARCHAR(10),
    "Customer ID" VARCHAR(30),"Customer Name" VARCHAR(30), Segment VARCHAR(30), Country VARCHAR(30),
    City VARCHAR(30),"State" VARCHAR(30),"Postal Code" VARCHAR(30), Region VARCHAR(30),"Product ID" VARCHAR(30),
    Category VARCHAR(30),"Sub-Category" VARCHAR(30),"Product Name" VARCHAR(100), Sales DOUBLE, Quantity INTEGER
    )
    INTO bulk.superstore (OrderID, CustomerID, OrderDate, Category, ProductName, Sales, Quantity)
    VALUES("Order ID","Customer ID","Order Date","Category","Product Name",Sales, Quantity)
    USING {"from":{"file":{"header":true}}}
    '''
exec_sql(sql)


In [ ]:
## Carga desde Base de datos Externos
df=pd.read_sql('select top 5 * from bulk.superstore',connection)
display(df)

## Carga desde tabla Externa

In [ ]:
 sql='''
    CREATE TABLE bulk.orders(
    id bigint,
    invoice VARCHAR(20),
    stockcode VARCHAR(20),
    description VARCHAR(100),
    quantity INTEGER,
    invoicedate Date,
    price NUMERIC,
    customerid VARCHAR(10)
    )
    '''
exec_sql(sql)

In [ ]:
sql=''' LOAD DATA FROM JDBC CONNECTION onlinestore
        TABLE "public"."orders"
        INTO bulk.orders(id, invoice, stockcode, description, quantity, invoicedate, price, customerid)
        VALUES(id, invoice, stockcode, description, quantity, invoicedate, price, customerid)
    '''
exec_sql(sql)

In [ ]:
## Carga desde Base de datos Externos
df=pd.read_sql('select top 5 * from bulk.orders',connection)
display(df)

## Notas adicionales de Carga Rapida

Las opciones siguientes permiten accelerar la carga massiva de datos. 
Construir los Indices al final de la carga es más eficiente:
LOAD BULK
LOAD %NOJOURN BULK


# Interoperabilidad: Carga con Record Mapper

Desde el portal de Gestión de IRIS, se puede usar el Record Mapper de Interoperabilidad para leer ficheros CSV, y generar los mensajes de interoperabilidad correspondientes

<img src="./img/recordmapper.png" style="height:300px">